# This is a systematic guide to data preprocessing for machine learning.

## Step 1: Load the Data

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = "your_data.csv"  # Replace with actual file path
data = pd.read_csv(file_path)

# Display basic info
print(data.head())  # Show the first few rows
print(data.info())  # Check for missing values and data types
